In [52]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [53]:
N_EPOCHS = 50
N_TRIALS = 3
BATCH_SIZE = 256

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [54]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

In [55]:
noise = 30
x_train = np.clip(x_train + np.random.normal(0, noise, x_train.shape), 0, 255) / 255
x_val = np.clip(x_val + np.random.normal(0, noise, x_val.shape), 0, 255) / 255
x_test = np.clip(x_test + np.random.normal(0, noise, x_test.shape), 0, 255) / 255

In [56]:
assessment_model = ClassifierModel(output_shape=10, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
assessment_model.fit(x_train, y_train, epochs=60, batch_size=BATCH_SIZE)

Epoch 1/60
28/28 [==============================] - 2s 44ms/step - loss: 2.5682 - accuracy: 0.1187
Epoch 2/60
28/28 [==============================] - 1s 45ms/step - loss: 2.2211 - accuracy: 0.1963
Epoch 3/60
28/28 [==============================] - 1s 51ms/step - loss: 2.1110 - accuracy: 0.2316
Epoch 4/60
28/28 [==============================] - 2s 54ms/step - loss: 2.0196 - accuracy: 0.2593
Epoch 5/60
28/28 [==============================] - 2s 58ms/step - loss: 1.9400 - accuracy: 0.2900
Epoch 6/60
28/28 [==============================] - 2s 56ms/step - loss: 1.8830 - accuracy: 0.3056
Epoch 7/60
28/28 [==============================] - 2s 56ms/step - loss: 1.8353 - accuracy: 0.3176
Epoch 8/60
28/28 [==============================] - 1s 48ms/step - loss: 1.8046 - accuracy: 0.3344
Epoch 9/60
28/28 [==============================] - 1s 49ms/step - loss: 1.7718 - accuracy: 0.3444
Epoch 10/60
28/28 [==============================] - 2s 54ms/step - loss: 1.7445 - accuracy: 0.3634
Epoch 11/

In [57]:
y_pred_assessment = assessment_model.predict(x_train, verbose=0)

losses_assessment = utils.calculate_loss_per_sample(y_train, y_pred_assessment, loss)
losses_proba = np.exp(losses_assessment) / sum(np.exp(losses_assessment))

In [58]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [ ]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
        last_best = 0
        n_same = 0
        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)
            samples_ids = np.random.choice(range(train_size), p=losses_proba, size=n_samples, replace=False)
            
            history = model.fit(
                x_train[samples_ids], y_train[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE, verbose=1,
            )
            
            n_same += 1
            if last_best < history.history["val_accuracy"][-1]:
                last_best = history.history["val_accuracy"][-1]
                n_same = 0
            
            if i >= 30 and n_same == 5:
                print(f"LAST EPOCH: {i}")
                break
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
        
        model_scores.append(accuracy)
    
    results[test_model] = model_scores

test_model_1


  0%|          | 0/3 [00:00<?, ?it/s]

28/28 [==============================] - 1s 31ms/step - loss: 1.3835 - accuracy: 0.4909 - val_loss: 1.4611 - val_accuracy: 0.4920
LAST EPOCH: 44
6/6 [==============================] - 0s 13ms/step - loss: 1.4364 - accuracy: 0.4680


 33%|███▎      | 1/3 [00:40<01:20, 40.15s/it]

19/28 [===================>..........] - ETA: 0s - loss: 1.4434 - accuracy: 0.4579

In [ ]:
results